### import packages

In [1]:
import os, sys
sys.path.append('..')
import numpy as np
import pandas as pd
from glob import glob 
from skimage.io import imread, imsave
from easi_fish import n5_metadata_utils as n5mu
from easi_fish import roi_prop, spot, intensity
import importlib
importlib.reload(spot)
importlib.reload(intensity)

<module 'easi_fish.intensity' from 'D:\\SWAP\\code\\EASI-FISH\\data_processing\\..\\easi_fish\\intensity.py'>

### Specify directories to data files

In [2]:
r1_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\"
r2_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\"

# lb_dir = "/LHA3_R3_medium/segmentation/LHA3_R3_medium-c2.tif"
# bad_roi='/test/bad_roi_list.npy'
# spot_dir = '/test/LHA3_R3_medium/spots/'
# out_dir = '/test/LHA3_R3_medium/spots/'
# img_dir='/test/LHA3_R3_medium/stitching/export.n5'


fix_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\export_sigma3.n5"
reg_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped"  # directory to the registered image
out_dir = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r1\\registration\\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\\warped\\testout-july29"  # where the output files should be saved 
lb_dir  = "D:\\SWAP\\Vincent\\lt171_FlpO\\gene_new_4tile\\outputs\\r2\\segmentation\\lt171_gene_4tile_r2-c3.tif"  # directory to the segmentation mask (tif format accepted here)

bad_roi = os.path.join(out_dir, "bad_roi_list.npy")
spot_dir = os.path.join(r2_dir, "spots")
spot_dir_r1 = os.path.join(r1_dir, "spots", "lt171_gene_4tile_r1-to-lt171_gene_4tile_r2")
out_dir = spot_dir
img_dir = fix_dir

print(bad_roi)
print(spot_dir)

D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\registration\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\warped\testout-july29\bad_roi_list.npy
D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots


### Load segmentation mask

In [3]:
%%time
### Get list of ROIs that are fully or partially outside the mask 
### Make sure to only include ROIs that are intact and in the overlapping regions across all rounds of FISH
lb=imread(lb_dir)
roi = np.max(lb)
print("number of ROIs in total:",roi)

bad_roi_list=np.load(bad_roi)
print('number of bad ROIs:', len(bad_roi_list))

number of ROIs in total: 13758
number of bad ROIs: 892
CPU times: total: 13.7 s
Wall time: 6.64 s


### Extract ROI properties
The roi_prop function exports the following: 
- ROI ID
- position (x, y, z in um, pre-expansion)
- size (volume, pre-expansion) 
- mionr_axis_length
- major_axis_length
- eccentricity =  mionr_axis_length/major_axis_length
- solidity = ROI volume/ROI convex hull

In [4]:
lb.shape # z, x,y 

(1019, 915, 916)

In [5]:
lb = lb[500:600,500:600,500:600]

In [6]:
%%time
df=roi_prop.roi_prop(lb)

C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1984118284  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width 10  Error-roundoff 1e-14  _one-merge 7.2e-14  _near-inside 3.6e-13
  Visible-distance 2.1e-14  U-max-coplanar 2.1e-14  Width-outside 4.1e-14
  _wide-facet 1.2e-13  _maxoutside 8.2e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     1     0
- p8(v3):     3     2     0
- p25(v2):    10     0     0
- p0(v1):     0     0     0

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet.

CPU times: total: 48.3 s
Wall time: 2.76 s


C:\Users\zeiss\anaconda3\envs\easifish\lib\site-packages\skimage\measure\_regionprops.py:395: UserWarning: Failed to get convex hull image. Returning empty image, see error message below:
QH6154 Qhull precision error: Initial simplex is flat (facet 1 is coplanar with the interior point)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1984118284  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _max-width  4  Error-roundoff 4.3e-15  _one-merge 3e-14  _near-inside 1.5e-13
  Visible-distance 8.5e-15  U-max-coplanar 8.5e-15  Width-outside 1.7e-14
  _wide-facet 5.1e-14  _maxoutside 3.4e-14

The input to qhull appears to be less than 3 dimensional, or a
computation has overflowed.

Qhull could not construct a clearly convex simplex from points:
- p1(v4):     0     0     2
- p7(v3):     1     0     4
- p3(v2):     1     0     0
- p0(v1):     0     0     1

The center point is coplanar with a facet, or a vertex is coplanar
with a neighboring facet. 

In [7]:
df

,roi,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
0,1.0,41.229921,31.799505,19.451946,79.007208,3.882587,26.592798,0.146001,0.787422
1,2.0,3.875844,23.812108,1.555209,265.993896,10.753584,24.829100,0.433104,0.896108
2,3.0,38.416707,41.567800,41.977212,443.649024,19.518692,24.459833,0.797990,0.937113
3,4.0,16.930890,36.248539,16.363112,1349.432448,26.958377,35.246973,0.764842,0.897983
4,5.0,18.313163,31.440374,43.722799,365.797152,12.974855,30.638046,0.423488,0.902236
...,...,...,...,...,...,...,...,...,...
57,58.0,40.003918,30.265155,0.308247,17.241168,3.235265,15.667296,0.206498,0.729323
58,59.0,7.900000,1.115774,1.490208,59.721984,8.114660,17.207684,0.471572,0.910569
59,60.0,1.291777,2.181560,39.471777,151.526760,9.737822,26.217210,0.371429,0.930169
60,61.0,41.422500,45.540000,10.120000,0.710976,0.000000,5.477226,0.000000,inf


### Remove bad ROIs and save the file

In [8]:
%%time
df_filtered=df.loc[~df['roi'].isin(bad_roi_list)]
df_filtered.roi=df_filtered.roi.astype(int)
df_filtered.to_csv(out_dir+'roi.csv')
df_filtered

CPU times: total: 2.75 s
Wall time: 938 ms


<timed exec>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,roi,z,y,x,area,minor_axis_length,major_axis_length,eccentricity,solidity
0,1,41.229921,31.799505,19.451946,79.007208,3.882587e+00,26.592798,1.460015e-01,0.787422
1,2,3.875844,23.812108,1.555209,265.993896,1.075358e+01,24.829100,4.331041e-01,0.896108
2,3,38.416707,41.567800,41.977212,443.649024,1.951869e+01,24.459833,7.979896e-01,0.937113
3,4,16.930890,36.248539,16.363112,1349.432448,2.695838e+01,35.246973,7.648423e-01,0.897983
4,5,18.313163,31.440374,43.722799,365.797152,1.297485e+01,30.638046,4.234883e-01,0.902236
5,6,3.468387,45.000242,21.145161,22.040256,5.069576e+00,12.484227,4.060785e-01,0.821192
6,7,33.308720,12.013752,18.997680,854.859768,2.402737e+01,29.253913,8.213385e-01,0.897462
7,8,19.084859,0.242958,42.728169,25.239648,2.807975e+00,17.285838,1.624437e-01,0.763441
8,9,1.035861,41.240594,43.788356,89.760720,8.701320e+00,25.099276,3.466761e-01,0.809944
9,10,15.134618,23.551570,38.701843,855.393000,2.410274e+01,29.004715,8.309938e-01,0.875876


### Lipofuscin subtraction (optional)

In [9]:
%%time
# Load spot files for lipofuscin subtraction
channel_1=np.loadtxt(os.path.join(spot_dir, 'spots_c0.txt'), delimiter=',')
channel_2=np.loadtxt(os.path.join(spot_dir, 'spots_c1.txt'), delimiter=',')
# Identify lipofuscin spots
c0,c1,pAind,pBind=spot.rm_lipofuscin(channel_1,channel_2)
# Remove lipofuscin spots
true_pos_c0 = np.delete(channel_1, pAind, axis=0)
true_pos_c1 = np.delete(channel_2, pBind, axis=0)

CPU times: total: 3.27 s
Wall time: 3.25 s


### spot extraction and save file

In [15]:
%%time
spotcount=spot.spot_counts(lb,spot_dir, verbose=False)
spotcount.to_csv(out_dir+'/spots_r2.csv')

Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c0.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c1.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c2.txt
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r2\spots\spots_c4.txt
CPU times: total: 20.1 s
Wall time: 20.1 s


In [16]:
%%time
spotcount=spot.spot_counts(lb,spot_dir_r1, verbose=False)
spotcount.to_csv(out_dir+'/spots_r1.csv')

Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c0_warped.txt
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c0_warped.txt line# 79374
Load: D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c1_warped.txt
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c1_warped.txt line# 59548
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c1_warped.txt line# 293582
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c1_warped.txt line# 313383
NaN found in D:\SWAP\Vincent\lt171_FlpO\gene_new_4tile\outputs\r1\spots\lt171_gene_4tile_r1-to-lt171_gene_4tile_r2\spots_c1_warped.txt line# 320791
NaN fo

### Intensity measurement and save file

In [14]:
%%time
df_intensity=intensity.measure_intensity(lb, img_dir,'c0', cropdim=(500,600,500,600,500,600))
df_intensity.to_csv(out_dir+'/intennsity_c0_r2.csv')

CPU times: total: 172 ms
Wall time: 168 ms


In [13]:
df_intensity

,roi,c0
0,362,93.434196
1,1314,93.571667
2,2167,93.942909
3,2944,93.400487
4,3254,93.980807
...,...,...
57,11418,91.804124
58,12209,91.910714
59,12387,92.777713
60,12406,91.500000
